# Interacting Particle Markov Chain Monte Carlo
## Introduction & Context

This notebook presents the interacting particle markov chain monte carlo algorithm from http://proceedings.mlr.press/v48/rainforth16.html by Rainforth et al. (Proceedings of the 33rd International Conference on Machine Learning,). This algorithm is an extension of the family of particleMarkov chain Monte Carlo algorithms originally proposed in https://www.stats.ox.ac.uk/~doucet/andrieu_doucet_holenstein_PMCMC.pdf by Andrieu et al. .

### Mathmatical context

We will focus on a Markovian model even if the algorithm is not limited to this type of models.

We have $x_t$ the states of our model and $y_t$ our observation following:
- $ x_t | x_{1:t−1} ∼ f_t(x_t|x_{1:t−1}) $ the transition model 
- $ y_t | x_{1:t} ∼ g_t ( y_t | x_{1:t}) $ the observation model
- $ x_1 ∼ \mu(\cdot)$

Our goal is to compute expectation values with respect to the posterior distribution $ p(x_{1:T}|y_{1:T}) \propto \mu(x_1)\prod\limits_{t=2}^{T}f_t(x_t | x_{1:t-1}) \prod\limits_{t=1}^{T} g_t ( y_t | x_{1:t} ) $

### Difficulties
As explained in Andrieu et al., Sequential Monte Carlo (SMC) and Markov Chain Monte Carlo (MCMC) methods are often unreliable when the proposal distributions that are used to explore the space are poorly chosen and/or if highly correlated variables are updated independently. Particle Markov Chain Monte Carlo (PMCMC) try to solve theses issues using SMC methods to construct efficient proposals for the MCMC sampler. 

Interacting particle Markov Chain Monte Carlo tries to increase the efficiency of PMCMC, and solve its issue with *path degeneracy*.
Whenever the ancestral lineage collapses at the early stages of the state sequence, the common ancestor is, by construction, guaranteed to be equal to the retained particle resulting in sample impoverishment, high correlation between the samples, and poor mixing of the Markov chain.

To make things more concrete let's dive in the algorithm!

## iPMCMC
### Algorithm

To explore the iPMCMC algorithm we must first explore SMC and CSMC, as iPMCMC uses both to generate efficient proposals of $x_{1:T}$ to the MCMC sampler.

#### Sequential Monte Carlo

The main idea of SMC is to generate at each time step a new position for each particle in our system, but instead of generating the next position of a particle with the particle past we take the past of any particle in our system with a discrete distribution based on the normalized weights of each one. 

We compute the weights of each particle at a given time with the following formula:

$$ w_{t}^{i} = \frac{g_t (y_t | x_{1:t}^{i}) f_t( x_{t}^{i}|x_{1:t-1}^{a_{t-1}^{i}} )}{ q_t(x_t^i | x_{1:t-1}^{a_{t-1}^{i}}) } $$

where $g_t$ is our observation model, $f_t$ our transition model and $q_t$ our proposal.

The resulting particle system approximates $p(x_{1:t}|y_{1:t})$

<img src="./images/algo1.PNG" width=500px></img>

#### Conditional Sequential Monte Carlo

CSMC is basically the same as SMC but fixing one particle trajectory. All other particles can use this fixed past but the particle itself is fixed.

<img src="./images/algo2.PNG" width=500px></img>

#### Interaction Particle Markov Chain Monte Carlo

IPMCMC combines both previous algorithms as MCMC sampler.
We have M workers, P of theses M workers being CSMC samplers and M-P SMC sampler.
At each step $r$ of the sampler all the workers generate their output according to the previously explained protocols.
Then for each CSMC worker we randomize its output between its own and any of the SMC workers, with probabilities:

Following the same notations as before, let's define:

$$ \hat{Z}_m = \prod\limits_{t=1}^{T}\frac{1}{N}\sum\limits_{i=1}^{N}w^i_{t, m} $$

and

$$ \zeta_{m}^{j} = \frac{\hat{Z}_m \mathbb{1}_{m \notin c_{1:P\backslash j}} }{\sum\limits_{n=1}^{M} \hat{Z}_n \mathbb{1}_{n \notin c_{1:P\backslash j}}}$$

to define properly the probality of *switch-in* of a given node:

$$ P(c_j = m| c_{1:P\backslash j} ) = \hat{\zeta}_{m}^{j} $$

We then choose the fixed trajectory of each CSMC worker with a weighted randomize choice over the new outputs outputs chosen previously.

$$ P(b_j = i| c_j ) = \bar{w}_{T, c_j}^{i} $$ and $$x_{j}'[r] = x_{c_j}^{b_j}$$

The output of our algorithm is the chains of all CSMC workers chosen fixed particles.
That gives us $R \times P$ trajectories.

<img src="./images/algo3.PNG" width=500px></img>

### Results

In [1]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm
from ipmcmc.generate_data import *
from ipmcmc.linear_gaussian_state_model import *
from ipmcmc.non_linear_gaussian_state_model import *
from ipmcmc.smc import *
from ipmcmc.csmc import *
from ipmcmc.ipmcmc import *
from ipmcmc.estimation import *    


We implemented the same models as those in the paper. Each one of them is implemented and generates observations and states in the two next cells

In [2]:
# 4.1. Linear Gaussian State Space Model
np.random.seed(420)
# Parameters
t_max = 50
n_particles = 50

r = R.from_rotvec(np.array([7*np.pi/10, 3*np.pi/10, np.pi/20]))
rotation_matrix = r.as_dcm()
scaling_matrix = 0.99*np.eye(3)
beta = np.random.dirichlet(np.ones(20)*0.2, 3).transpose()
alpha = scaling_matrix@rotation_matrix
t_max = 50
mu = np.array([0, 1, 1])
start_var = 0.1*np.eye(3)
omega = np.eye(3)
sigma = 0.1*np.eye(20)



l_transition_model = [LinearMu(default_mean=mu, default_cov=start_var)]+[LinearTransition(
    default_mean=np.zeros(3), default_cov=omega, default_alpha=alpha) for t in range(1, t_max)]
l_proposals = [LinearMu(default_mean=mu, default_cov=start_var)]+[LinearProposal(
    default_mean=np.zeros(3), default_cov=omega, default_alpha=alpha) for t in range(1, t_max)]
l_observation_model = [LinearObservation(default_mean=np.zeros(
    20), default_cov=sigma, default_beta=beta) for t in range(0, t_max)]

# If we want to change the parameters
assert np.all(np.linalg.eigvals(start_var) > 0)
assert np.all(np.linalg.eigvals(omega) > 0)
assert np.all(np.linalg.eigvals(sigma) > 0)

l_states, l_observations = linear_gaussian_state_space(
    t_max=t_max, mu=mu, start_var=start_var, transition_var=omega, noise_var=sigma,
    transition_coeffs=alpha, observation_coeffs=beta)

In [3]:
# 4.2. Nonlinear State Space Model
np.random.seed(420)
nl_mu = 0
start_std = np.sqrt(5)
omega = np.sqrt(10)
sigma = np.sqrt(10)

nl_transition_model = [NonLinearMu(default_mean=nl_mu, default_std=start_std)]+[NonLinearTransition(
    default_mean=0, default_std=omega) for t in range(1, t_max)]
nl_proposals = [NonLinearMu(default_mean=nl_mu, default_std=start_std)]+[
    NonLinearProposal(default_mean=0, default_std=omega) for t in range(1, t_max)]
nl_observation_model = [NonLinearObservation(
    default_mean=0, default_std=sigma) for t in range(0, t_max)]

nl_states, nl_observations = nonlinear_gaussian_state_space(
    t_max=t_max, mu=nl_mu, start_std=start_std, transition_std=omega, noise_std=sigma)

In [4]:
# ipmcmc run: works with both linear and non-linear models.
# It is pretty long to run, longer for the linear model which has 3-dimensional states.
# For the linear model, each MCMC step take approximately 90 secs, and 80 secs for 
# the non-linear, on our computers.

n_nodes = 32
n_conditional_nodes = 16
n_steps = 10

linear= True

if linear:
    print('init_conditional_traj')
    
    init_conditional_traj = np.zeros((n_conditional_nodes, t_max, len(mu)))
    for i in tqdm(range(n_conditional_nodes)):
        
        particles, weights, _ = smc(l_observations, n_particles,
                              l_transition_model, l_proposals, l_observation_model)
        b_j = np.argmax(weights[-1])
        init_conditional_traj[i] = particles[:,b_j]

    print('running ipmcmc')
    particles, conditional_traj, weights, conditional_indices, zetas = ipmcmc(
        n_steps, n_nodes, n_conditional_nodes, l_observations, n_particles, init_conditional_traj,
        l_proposals, l_transition_model, l_observation_model)

else:
    print('init_conditional_traj')
    
    init_conditional_traj = np.zeros((n_conditional_nodes, t_max, 1))
    for i in tqdm(range(n_conditional_nodes)):
        particles, weights, _ = smc(nl_observations, n_particles,
                              nl_transition_model, nl_proposals, nl_observation_model)
        b_j = np.argmax(weights[-1])
        init_conditional_traj[i] = particles[:,b_j]

    print('running ipmcmc')
    particles, conditional_traj, weights, conditional_indices, zetas = ipmcmc(
        n_steps, n_nodes, n_conditional_nodes, nl_observations, n_particles, init_conditional_traj,
        nl_proposals, nl_transition_model, nl_observation_model)
        

  0%|          | 0/16 [00:00<?, ?it/s]

init_conditional_traj


  0%|          | 0/10 [00:00<?, ?it/s]

running ipmcmc


100%|██████████| 10/10 [08:05<00:00, 48.57s/it]


### Evaluation

There are several ways to see if our sampler works correctly. 

#### Estimation
One one them if to estimate some parameters of the distribution, function of our retained particles from the MCMC steps $\{\mathbf{x}_{1:P}'[r]\}_{r\in\{1,\dots,R\}}$:

$$\mathbb{E}[f(\mathbf{x})] \approx \frac{1}{RP}\sum\limits_{r=1}^R\sum\limits_{j=1}^P f(\mathbf{x}_j'[r])$$

But in this case we would not be using all the particles generated during the rum of our iPMCMC sample. Instead we can use an estimator that uses all the particles and has a smaller variance in general. This procedure called Rao-Blackwellisation comes to replace $f(\mathbf{x}_j'[r])$ in the above expression by

$$\mathbb{E}_{c_j|c_{1:P\backslash j}} [\mathbb{E}_{b_j|c_j}[\mathbf{x}_{1:P}'[r]]] = \sum\limits_{m=1}^M\hat{\zeta}_m^j\sum\limits_{i=1}^N \bar{w}_{T,m}^if(\mathbf{x}^i_m)$$


That is what we did to estimate the latent variables means, so $f$ is the identity function in the above formulas. We computed the error by averaging over all dimensions, since we can find the ground truth solution by running a kalman filter followed by a RTS smoothing on the observations.

Since we did not have that many MCMC steps due to the computation time, we did not find the same results as in the paper, but the paper show pretty clearly how iPMCMC beats every other algorithm. If we condider the MSE function of MCMC step, taking the average over all step in the state sequence, iPMCMC beats the other algorithm at approximately 100 iterations. If we consider the MSE function of step in the state sequence for the last iteration of the MCMC algo, the iPMCMC has the best performance at the begining of the sequence, and then as similar performances to the other algorithms.

In [5]:
# Mean estimation for the linear model, using kalman filter and rts smoother as ground truth
# Make sure that the particles used are the one generated during a run of the ipmcmc sampler
# for the linear model

true_means, true_covs = compute_ground_truth(l_observations, mu, start_var, alpha, omega, beta, sigma)

rao_black_traj = rao_blackwellisation(particles, weights, zetas, n_conditional_nodes)

errors_function_of_mcmc_step = []
errors_function_of_state_step = []
for r in range(1, (n_steps+1)):
    errors_function_of_mcmc_step.append(compute_error(rao_black_traj, true_means, r))

for t in range(1, (t_max+1)):
    errors_function_of_state_step.append(compute_error(rao_black_traj, true_means, state_step=t))

#### Plotting the particles histogram

For the non linear state space model, we can not compute analytically the ground truth, so we need to find other solutions. One of them is to plot the histogram of the generated particles at a given step in the state sequence. Once again, we can see that iPMCMC outperform early the other algorithms, and has the same performances for later steps. For the ground truth we ran a kernel density estimator on combined samples from a small number of independant SMC sweeps, each with $10^7$ particles. 